In [8]:
from __future__ import print_function;
import sys;
import findspark
findspark.init('/home/imran/spark')


from pyspark.sql import SparkSession;
from pyspark.sql.types import StructType, StructField, StringType, IntegerType;
from pyspark.sql.functions import regexp_replace, ltrim, rtrim;

filepath = "/home/imran/py/PepoleEDA/data/ConfLongDemo_JSI.txt";

spark = SparkSession.builder.appName("Parse DF Using UDF").getOrCreate();

In [16]:
df = spark.read.csv(filepath, inferSchema=True, ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True, maxColumns=100
                   , timestampFormat='dd.MM.yyyy HH:mm:ss:SSS' );

In [20]:
print(df.schema);
df.select('_c3').distinct().take(1)

StructType(List(StructField(_c0,StringType,true),StructField(_c1,StringType,true),StructField(_c2,LongType,true),StructField(_c3,TimestampType,true),StructField(_c4,DoubleType,true),StructField(_c5,DoubleType,true),StructField(_c6,DoubleType,true),StructField(_c7,StringType,true)))


[Row(_c3=datetime.datetime(2009, 5, 27, 14, 3, 25, 453000))]

In [37]:
df = spark.read.csv(filepath, inferSchema=True, ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True, maxColumns=100);
df.select("_c3").show(1);
print(df.schema);

+--------------------+
|                 _c3|
+--------------------+
|27.05.2009 14:03:...|
+--------------------+
only showing top 1 row

StructType(List(StructField(_c0,StringType,true),StructField(_c1,StringType,true),StructField(_c2,LongType,true),StructField(_c3,StringType,true),StructField(_c4,DoubleType,true),StructField(_c5,DoubleType,true),StructField(_c6,DoubleType,true),StructField(_c7,StringType,true)))


In [48]:
from datetime import datetime;
def func_formatdate(column):
    print(column)
    return datetime.strptime(column, '%d.%m.%Y %H:%M:%S:%f');
#     return column;

print(func_formatdate('27.05.2009 14:03:25:127'));


27.05.2009 14:03:25:127
2009-05-27 14:03:25.127000


In [52]:
df = df.withColumn('formatDate', func_formatdate(df._c3))
df.select("formatDate").show(1)


Column<_c3>


TypeError: strptime() argument 1 must be string, not Column